# Best build by weighted relic effects

Given a list of desired relic effects to which we give a desirability value, find combinations of available Vessels and relics that maximizes the value of effects we want in our build.

In [1]:
import itertools
import yaml

from collections import Counter

from items import Vessel, Relic

In [2]:
ironeye_vessels = [
    Vessel(("Yellow", "Green", "Green")),
    Vessel(("Red", "Blue", "Yellow")),
    Vessel(("Red", "Green", "White"))
]

In [3]:
inventory = []

with open("relic_inventory.yml", "r") as f:
    db = yaml.safe_load(f)
    for entry in db:
        inventory.append(Relic(entry['color'], entry['effects']))

In [4]:
inventory

[Relic(color='Red', effects=['Stamina Recovers with Each Successful Attack', 'Dexterity +1', 'Lightning Attack Power Up +2']),
 Relic(color='Red', effects=['Improved Invisibility Sorcery', 'Dexterity +2']),
 Relic(color='Red', effects=['Improved Heavy Thrusting Sword Attach Power']),
 Relic(color='Red', effects=['Improved Attack Power with 3+ Axes Equipped', 'Improved Godslayer Incantations', "Changes compatible armament's skill to Chilling Mist at start of expedition"]),
 Relic(color='Red', effects=['[Raider] Damage taken while using Character Skill improves attack power and stamina', 'Strength +3', 'Magic Attack Power Up +1']),
 Relic(color='Red', effects=['[Revenant] Trigger ghostflame explosion during Ultimate Art activation', "Changes compatible armament's skill to Hoarfrost"]),
 Relic(color='Yellow', effects=['Starting armament inflics frost', 'Magic Attack Power Up +1']),
 Relic(color='Yellow', effects=['[Ironeye] +1 additional Character Skill use', 'Starting armament inflicts b

In [5]:
with open("effect_preferences.yml", "r") as f:
    effect_preferences = yaml.safe_load(f)

In [6]:
effect_preferences["Ironeye"]

{'Starting armament Inflicts Frostbite': 100,
 '[Ironeye] Art Charge Activation Adds Poison Effect': 100,
 '[Ironeye] +1 additional Character Skill use': 80,
 'Attack Power Up when Facing Frostbite-afflicted Enemy': 80,
 'Attack Power Up when Facing Poison-afflicted Enemy': 80,
 'Stamina Recovers with Each Successful Attack +1': 80,
 'Stamina Recovers with Each Successful Attack': 75,
 'Character Skill Cooldown Reduction +3': 60,
 'Arcane +3': 60,
 'Improved Bow Attack Power': 60,
 'Character Skill Cooldown Reduction +2': 55,
 'Arcane +2': 55,
 'Character Skill Cooldown Reduction +1': 20,
 'Arcane +1': 20}

In [7]:
def score_relics_in_inventory(inventory, effect_preferences):
    for relic in inventory:
        relic.score = 0
        for effect in relic.effects:
            relic.score += effect_preferences.get(effect, 0)
    return inventory

In [8]:
score_relics_in_inventory(inventory, effect_preferences["Ironeye"])

[Relic(color='Red', effects=['Stamina Recovers with Each Successful Attack', 'Dexterity +1', 'Lightning Attack Power Up +2']),
 Relic(color='Red', effects=['Improved Invisibility Sorcery', 'Dexterity +2']),
 Relic(color='Red', effects=['Improved Heavy Thrusting Sword Attach Power']),
 Relic(color='Red', effects=['Improved Attack Power with 3+ Axes Equipped', 'Improved Godslayer Incantations', "Changes compatible armament's skill to Chilling Mist at start of expedition"]),
 Relic(color='Red', effects=['[Raider] Damage taken while using Character Skill improves attack power and stamina', 'Strength +3', 'Magic Attack Power Up +1']),
 Relic(color='Red', effects=['[Revenant] Trigger ghostflame explosion during Ultimate Art activation', "Changes compatible armament's skill to Hoarfrost"]),
 Relic(color='Yellow', effects=['Starting armament inflics frost', 'Magic Attack Power Up +1']),
 Relic(color='Yellow', effects=['[Ironeye] +1 additional Character Skill use', 'Starting armament inflicts b

In [9]:
def get_relic_combinations(vessel, inventory):
    possible_relics = [
        [relic for relic in inventory if relic.color == slot_color or slot_color == "white"]
        for slot_color in vessel.slot_colors
    ]
    return [
        vessel for vessel in itertools.product(*possible_relics)
        if all(value == 1 for value in Counter(vessel).values())
    ]

In [10]:
possible_relic_combinations = get_relic_combinations(ironeye_vessels[0], inventory)

In [19]:
def score_relic_combinations(relic_combinations):
    scores = {
        relic_combination: sum([relic.score for relic in relic_combination])
        for relic_combination in relic_combinations
    }
    return {
        k: v
        for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)
    }

In [20]:
score_relic_combinations(possible_relic_combinations)

{(Relic(color='Yellow', effects=['[Ironeye] +1 additional Character Skill use', 'Starting armament inflicts blood loss']),
  Relic(color='Green', effects=['[Ironeye] Art Charge Activation Adds Poison Effect', 'Arcane +3']),
  Relic(color='Green', effects=['Draw Enemy Attention while Guarding', 'Switching Weapons Adds an Affinity Attack', 'Arcane +3'])): 300,
 (Relic(color='Yellow', effects=['[Ironeye] +1 additional Character Skill use', 'Starting armament inflicts blood loss']),
  Relic(color='Green', effects=['Draw Enemy Attention while Guarding', 'Switching Weapons Adds an Affinity Attack', 'Arcane +3']),
  Relic(color='Green', effects=['[Ironeye] Art Charge Activation Adds Poison Effect', 'Arcane +3'])): 300,
 (Relic(color='Yellow', effects=['[Ironeye] +1 additional Character Skill use', 'Starting armament inflicts blood loss']),
  Relic(color='Green', effects=['Improved Hammer Attack Power', 'Vigor +2', 'Endurance +1']),
  Relic(color='Green', effects=['[Ironeye] Art Charge Activat